In [638]:
# Importing necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
import pandas as pd
import re
# Loading the Boston housing dataset
# housing = fetch_openml(name="house_prices", as_frame=True)



In [639]:
# import pandas as pd

# # قراءة ملف XLSX
# df = pd.read_excel('data .xlsx')

# # حفظ البيانات في ملف CSV مع الترميز المناسب
# df.to_csv('data_with_arabic.csv', encoding='utf-8-sig', index=False)


In [640]:
data =  pd.read_csv('data_with_arabic.csv',encoding='utf-8-sig')

In [641]:
# data=data.drop_duplicates()

In [642]:
# df = pd.DataFrame(data)

In [643]:
# df.to_csv("data_new.csv", index=False)

In [644]:
data

                                              Table 1  CDR(OD)  CDR(OS)  \
0                          زرق أيمن تم معالجته جراحيا      0.9      0.5   
1                                           زرق متقدم      0.8      NaN   
2                                         لا يوجد زرق      0.4      0.5   
3                               زرق مزدوج أشد بالأيمن      0.7      0.7   
4                           زرق بالعينتين أشد بالأيمن      NaN      NaN   
5                           زرق أيسر وتم علاجه جراحيا      0.8      0.9   
6                                            زرق أيسر      0.3  0.8-0.9   
7                                       مريض غير زرقي      0.7      0.8   
8                           زرق بالعينتين أشد بالأيمن      0.8      0.4   
9                                            زرق أيسر      0.3      0.3   
10                                    زرقي وبالعينتين       10      0.8   
11                             مريض زرقي مضبوط دوائيا      NaN      NaN   
12  مريضة زرقية، أذية زرق

In [645]:
# Create a new DataFrame to store the split rows
split_rows = []

# Iterate through each row of the original DataFrame
for index, row in data.iterrows():
    # Create a copy of the row for the left eye (OS)
    os_row = row.copy()
    os_row['Eye'] = 'OS'  # Left Eye
    os_row['VA'] = row['VA(OS)']
    os_row['CDR'] = row['CDR(OS)']
    os_row['Diagnosis'] = row['Diagnosis(OS)']
    os_row['IOP'] = row['IOP(OS)']
    split_rows.append(os_row)
    
    # Create a copy of the row for the right eye (OD)
    od_row = row.copy()
    od_row['Eye'] = 'OD'  # Right Eye
    od_row['VA'] = row['VA(OD)']
    od_row['CDR'] = row['CDR(OD)']
    od_row['Diagnosis'] = row['Diagnosis(OD)']
    od_row['IOP'] = row['IOP(OD)']
    split_rows.append(od_row)

# Create a new DataFrame from the split rows
split_df = pd.DataFrame(split_rows)

# Drop the unnecessary columns
split_df.drop(columns=['VA(OS)', 'VA(OD)', 'CDR(OS)', 'CDR(OD)', 'Diagnosis(OS)', 'Diagnosis(OD)', 'IOP(OS)', 'IOP(OD)'], inplace=True)

# Display the new DataFrame
print(split_df)


                                              Table 1 Spectacles  Age Gender  \
0                          زرق أيمن تم معالجته جراحيا    لا يوجد   64    ذكر   
0                          زرق أيمن تم معالجته جراحيا    لا يوجد   64    ذكر   
1                                           زرق متقدم       يوجد   62    ذكر   
1                                           زرق متقدم       يوجد   62    ذكر   
2                                         لا يوجد زرق    لا يوجد   48   أنثى   
2                                         لا يوجد زرق    لا يوجد   48   أنثى   
3                               زرق مزدوج أشد بالأيمن       يوجد   60    ذكر   
3                               زرق مزدوج أشد بالأيمن       يوجد   60    ذكر   
4                           زرق بالعينتين أشد بالأيمن       يوجد   58   أنثى   
4                           زرق بالعينتين أشد بالأيمن       يوجد   58   أنثى   
5                           زرق أيسر وتم علاجه جراحيا       يوجد   65   أنثى   
5                           زرق أيسر وتم

In [646]:
split_df

                                              Table 1 Spectacles  Age Gender  \
0                          زرق أيمن تم معالجته جراحيا    لا يوجد   64    ذكر   
0                          زرق أيمن تم معالجته جراحيا    لا يوجد   64    ذكر   
1                                           زرق متقدم       يوجد   62    ذكر   
1                                           زرق متقدم       يوجد   62    ذكر   
2                                         لا يوجد زرق    لا يوجد   48   أنثى   
2                                         لا يوجد زرق    لا يوجد   48   أنثى   
3                               زرق مزدوج أشد بالأيمن       يوجد   60    ذكر   
3                               زرق مزدوج أشد بالأيمن       يوجد   60    ذكر   
4                           زرق بالعينتين أشد بالأيمن       يوجد   58   أنثى   
4                           زرق بالعينتين أشد بالأيمن       يوجد   58   أنثى   
5                           زرق أيسر وتم علاجه جراحيا       يوجد   65   أنثى   
5                           زرق أيسر وتم

In [647]:
# # Set Diagnosis to 1 where CDR > 0.5 and Diagnosis is NaN
# split_df.loc[(split_df['CDR'] > 0.5) & (split_df['Diagnosis'].isnull()), 'Diagnosis'] = 1

# # Set Diagnosis to 0 where CDR <= 0.5 and Diagnosis is NaN
# split_df.loc[((split_df['CDR'] <= 0.5) | (split_df['CDR'].isnull())) & (split_df['Diagnosis'].isnull()), 'Diagnosis'] = 0


In [648]:
gender_mapping = {'ذكر': 0 , 'أنثى': 1}
split_df['Gender'] = split_df['Gender'].map(gender_mapping)

eye_mapping = {'OS': 0, 'OD': 1}
split_df['Eye'] = split_df['Eye'].map(eye_mapping)

eye_mapping = {'لا يوجد': 0, 'يوجد': 1}
split_df['Spectacles'] = split_df['Spectacles'].map(eye_mapping)



In [649]:
cf_array = []
for i in range(len(split_df['VA'])):
    # Check if the value is a string and if it contains 'cf' or 'HM' (case-insensitive)
    if isinstance(split_df['VA'].iloc[i], str) and ('cf' in split_df['VA'].iloc[i].lower() or 'HM' in split_df['VA'].iloc[i]):
        print(split_df['VA'].iloc[i])
        cf_array.append(split_df['VA'].iloc[i])


Cf 0.5 m
أمام العين cf
HM
cf 2 m
cf 2 m
أمام العين cf
HM
HM


In [650]:
for i in range(len(split_df['VA'])):
    for j in range(len(cf_array)):
        if split_df['VA'].iloc[i] == cf_array[j]:
            split_df['VA'].iloc[i] = 9
    

C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1394833798.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['VA'].iloc[i] = 9
C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1394833798.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['VA'].iloc[i] = 9
C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1394833798.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['VA'].iloc[i] = 9
C:\Users\tony\AppData\Loc

In [651]:
for i in range(len(split_df['VA'])):
    if split_df['VA'].iloc[i] == '10':
        # print(split_df['VA'].iloc[i])
        split_df['VA'].iloc[i] = 1.0

C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1274017944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['VA'].iloc[i] = 1.0
C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1274017944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['VA'].iloc[i] = 1.0
C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1274017944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['VA'].iloc[i] = 1.0
C:\Users\tony\AppDa

In [652]:
def get_bigger_number(phrase):
    numbers = re.findall(r'\d+\.\d+', phrase)  # Match any decimal number
    if len(numbers) == 2:  # If two numbers are found (as in the pattern 0.4-0.5)
        bigger_number = max(map(float, numbers))  # Convert numbers to float and get the maximum
        return bigger_number

In [653]:
for i in range(len(split_df['VA'])):
    if isinstance(split_df['VA'].iloc[i], str) and '-' in split_df['VA'].iloc[i]:
        bigger_number = get_bigger_number(split_df['VA'].iloc[i])
        split_df['VA'].iloc[i] = bigger_number

C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1983676122.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['VA'].iloc[i] = bigger_number
C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1983676122.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['VA'].iloc[i] = bigger_number
C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1983676122.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['VA'].iloc[i] = big

In [654]:
for i in range(len(split_df['CDR'])):
    if isinstance(split_df['CDR'].iloc[i], str) and '-' in split_df['CDR'].iloc[i]:
        bigger_number = get_bigger_number(split_df['CDR'].iloc[i])
        split_df['CDR'].iloc[i] = bigger_number

C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1389035994.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['CDR'].iloc[i] = bigger_number
C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1389035994.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['CDR'].iloc[i] = bigger_number
C:\Users\tony\AppData\Local\Temp\ipykernel_2256\1389035994.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_df['CDR'].iloc[i] = 

In [655]:
split_df

                                              Table 1  Spectacles  Age  \
0                          زرق أيمن تم معالجته جراحيا         0.0   64   
0                          زرق أيمن تم معالجته جراحيا         0.0   64   
1                                           زرق متقدم         1.0   62   
1                                           زرق متقدم         1.0   62   
2                                         لا يوجد زرق         0.0   48   
2                                         لا يوجد زرق         0.0   48   
3                               زرق مزدوج أشد بالأيمن         1.0   60   
3                               زرق مزدوج أشد بالأيمن         1.0   60   
4                           زرق بالعينتين أشد بالأيمن         1.0   58   
4                           زرق بالعينتين أشد بالأيمن         1.0   58   
5                           زرق أيسر وتم علاجه جراحيا         1.0   65   
5                           زرق أيسر وتم علاجه جراحيا         1.0   65   
6                                     

In [656]:
split_df.drop('Name', axis=1, inplace=True)

# Create a new "ID" column with sequential numbers starting from 1
split_df['ID'] = range(1, len(split_df) + 1)

In [657]:
X_data = split_df.drop(['Diagnosis','Table 1','Gender','id'],axis=1)
y_data = split_df['Diagnosis']

In [658]:
y_data

0     0
0     1
1     1
1     1
2     0
2     0
3     1
3     1
4     1
4     1
5     1
5     0
6     1
6     0
7     0
7     0
8     1
8     1
9     1
9     0
10    1
10    1
11    1
11    1
12    0
12    1
13    1
13    1
14    1
14    1
15    0
15    1
16    0
16    0
17    1
17    1
18    1
18    0
19    0
19    0
20    0
20    0
21    0
21    0
22    1
22    1
23    1
23    1
24    1
24    1
25    0
25    0
26    0
26    0
27    1
27    1
28    0
28    0
29    1
29    0
30    0
30    0
31    0
31    0
32    1
32    1
33    0
33    1
34    1
34    1
35    0
35    1
36    0
36    1
37    1
37    0
38    0
38    1
39    1
39    1
40    0
40    1
41    0
41    0
42    1
42    0
43    1
43    1
44    0
44    0
Name: Diagnosis, dtype: int64

In [659]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=44, shuffle=True)


In [660]:
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


In [661]:
KNeighborsRegressorModel = KNeighborsRegressor(n_neighbors=3, weights='uniform', algorithm='auto')
KNeighborsRegressorModel.fit(X_train_imputed, y_train)

KNeighborsRegressor(n_neighbors=3)

In [662]:
print('KNeighborsRegressorModel Train Score is : ', KNeighborsRegressorModel.score(X_train_imputed, y_train))
print('KNeighborsRegressorModel Test Score is : ', KNeighborsRegressorModel.score(X_test_imputed, y_test))


KNeighborsRegressorModel Train Score is :  0.4593241551939924
KNeighborsRegressorModel Test Score is :  -1.5714285714285712


In [663]:
y_pred = KNeighborsRegressorModel.predict(X_test_imputed)
print('Predicted Value for KNeighborsRegressorModel is : ', y_pred[:10])

# Calculating mean absolute error
MAEValue = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
print('Mean Absolute Error Value is : ', MAEValue)

# Calculating mean squared error
MSEValue = mean_squared_error(y_test, y_pred, multioutput='uniform_average')
print('Mean Squared Error Value is : ', MSEValue)

# Calculating median squared error
MdSEValue = median_absolute_error(y_test, y_pred)
print('Median Squared Error Value is : ', MdSEValue)

Predicted Value for KNeighborsRegressorModel is :  [0.66666667 0.66666667 0.66666667 0.33333333 1.         0.66666667
 1.         0.66666667 1.        ]
Mean Absolute Error Value is :  0.5925925925925926
Mean Squared Error Value is :  0.4444444444444444
Median Squared Error Value is :  0.6666666666666666


In [664]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.io import loadmat
from sklearn import svm

In [665]:
X_data = split_df.drop('Diagnosis',axis=1)
y_data = split_df['Diagnosis']

In [666]:
sns.set_context('notebook')
sns.set_style('white')
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)

In [667]:
def plotData(X , y ,S):
    pos= (y==1).ravel()
    neg= (y==0).ravel()

    plt.scatter(X[pos,0.0],X[pos,1.0],s=S,c='b',marker='+',linewidths=1)
    plt.scatter(X[neg,0.0],X[neg,1.0],s=S,c='r',marker='o',linewidths=1)
def plot_svc(svc, X, y, h=0.02, pad=0.25):
    x_min, x_max = X[:, 0].min()-pad, X[:, 0].max()+pad
    y_min, y_max = X[:, 1].min()-pad, X[:, 1].max()+pad
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.2)

    plotData(X, y, 6)
    # plt.scatter(X[:, 0], X[:, 1], s=70, c=y, cmap=plt.cm.Paired)
    # Support vectors indicated in plot by vertical lines
    sv = svc.support_vectors_
    plt.scatter(sv[:, 0], sv[:, 1], c='y', marker='|', s=100, linewidths='5')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()
    
    print('Number of support vectors: ', svc.support_size)

In [668]:
plotData(X_data,y_data,10)

InvalidIndexError: (array([False,  True,  True,  True, False, False,  True,  True,  True,
        True,  True, False,  True, False, False, False,  True,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True, False, False,  True,  True,
        True, False, False, False, False, False, False, False,  True,
        True,  True,  True,  True,  True, False, False, False, False,
        True,  True, False, False,  True, False, False, False, False,
       False,  True,  True, False,  True,  True,  True, False,  True,
       False,  True,  True, False, False,  True,  True,  True, False,
        True, False, False,  True, False,  True,  True, False, False]), 0.0)

In [ ]:
clf = svm.SVC(C=1.0,kernel='linear')
clf = svm.SVC(C=1.0,kernel='rbf',gamma=6) # cerve كيرفيل
# gamma مدى الليونة
clf = svm.SVC(C=1.0,kernel='ploy',degree= 3,gamma=10)